In [ ]:
import numpy as np
import matplotlib.pylab as plt

# 5. 오차역전파법

앞 장에서는 신경망의 가중치 매개변수의 기울기는 수치 미분을 사용해서 구함  
수치 미분은 시간이 오래 걸린다는 단점  
이번 장에서는 가중치 매개변수의 기울기를 효율적으로 계산하는 오차역전파법을 다룸  
오차역전파법을 이해하는 방법 두 가지: 수식, 계산 그래프

## 5.1 계산 그래프

<code>계산 그래프(computational graph)</code>: 계산 과정을 그래프로 나타낸 것, 노드(node)와 에지(edge)로 표현

### 5.1.1 계산 그래프로 풀다

ex. 현빈 군은 슈퍼에서 1개에 100원인 사과를 2개 샀습니다. 이때 지불 금액을 구하세요. 단, 소비세가 10% 부과됩니다.

<img width="601" alt="fig 5-2" src="https://user-images.githubusercontent.com/77653353/194425362-70f6152d-7220-4027-8207-2ae909133086.png">


<code>순전파(forward propagation)</code>: 계산을 왼쪽에서 오른쪽으로 진행하는 단계  
<code>역전파(backward propagation)</code>: 계산을 오른쪽에서 왼쪽으로 반대로 진행하는 단계

### 5.1.2 국소적 계산

계산 그래프는 <code>국소적 계산</code>에 집중함  
전체 계산이 아무리 복잡하더라도 각 단계에서 하는 일은 해당 노드의 국소적 계산

<img width="643" alt="fig 5-4" src="https://user-images.githubusercontent.com/77653353/194426040-1b161e9b-348a-4ae7-8bc4-77543d484943.png">

위의 그림에서는 여러 식품을 구입(복잡한 계산)을 거쳐 총 금액이 4,000원이 되었는데  
사과와 다른 물품 값을 더하는 계산(4,000 + 200 = 4,200)은  
4,000이라는 숫자가 어떻게 계산되었느냐와는 상관없이, 단지 두 숫자를 더하면 된다는 것

### 5.1.3 왜 계산 그래프로 푸는가?

계산 그래프를 사용하는 가장 큰 이유는 역전파를 통해 <code>미분</code>을 효율적으로 계산할 수 있다는 점!

맨 위의 예시에서  
만약 사과 가격이 오르면 최종 금액에 어떤 영향을 끼치는지를 알고 싶다고 해보자  
이는 '사과 가격에 대한 지불 금액의 미분'을 구하는 문제에 해당됨  
사과 값을 $x$, 지불 금액을 $L$이라 했을 때 $\frac{\partial{L}}{\partial{x}}$를 구하는 것

<img width="608" alt="fig 5-5" src="https://user-images.githubusercontent.com/77653353/194427222-dbe08b8b-4e0e-4d25-9e89-ea908a9fe499.png">

역전파는 오른쪽에서 왼쪽으로 '1 → 1.1 → 2.2' 순으로 미분 값을 전달함  
사과가 1원 오르면 최종 금액은 2.2원 오른다는 뜻

## 5.2 연쇄법칙

국소적 미분을 전달하는 원리는 <code>연쇄법칙(chain rule)</code>에 따른 것

### 5.2.1 계산 그래프의 역전파

<img width="282" alt="fig 5-6" src="https://user-images.githubusercontent.com/77653353/194427676-a92a8aa2-b16b-474b-83b1-3e375243ab16.png">

역전파의 계산 절차는 신호 $E$에 노드의 국소적 미분 $\frac{\partial{y}}{\partial{x}}$을 곱한 후 다음 노드로 전달하는 것

### 5.2.2 연쇄법칙이란?

연쇄법칙을 설명하려면 우선 합성 함수부터  
<code>합성 함수</code>: 여러 함수로 구성된 함수

ex. $z = (x+y)^2$이라는 식은 아래처럼 두 개의 식으로 구성됨

$$ z = t^2 $$
$$ t = x + y $$

'합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다' 가 연쇄법칙의 원리

수식으로 쓰면

$$ \frac{\partial{z}}{\partial{x}} = \frac{\partial{z}}{\partial{t}} \frac{\partial{t}}{\partial{x}} $$

$$ \frac{\partial{z}}{\partial{t}} = 2t $$  
$$ \frac{\partial{t}}{\partial{x}} = 1 $$  
$$ \frac{\partial{z}}{\partial{x}} = 2t \cdot 1 = 2(x+y) $$

### 5.2.3 연쇄법칙과 계산 그래프

위의 연쇄법칙을 계산 그래프로 나타내면

<img width="466" alt="fig 5-7" src="https://user-images.githubusercontent.com/77653353/194429186-34492d6f-3168-4762-bde2-ff5ace04d458.png">

## 5.3 역전파

### 5.3.1 덧셈 노드의 역전파

$z = x + y$ 라는 식이 있다면

$$ \frac{\partial{z}}{\partial{x}} = 1 $$  
$$ \frac{\partial{z}}{\partial{y}} = 1 $$

왼쪽이 순전파, 오른쪽이 역전파

<img width="651" alt="fig 5-9" src="https://user-images.githubusercontent.com/77653353/194429956-0449676d-80a9-4e1d-bca8-e19c1661038e.png">

위의 그림에서 상류에서 전해진 미분이 $\frac{\partial{L}}{\partial{z}}$이라고 한다면  
덧셈 노드의 역전파는 <code>입력된 값을 그대로 다음 노드로</code> 보내게 됨

### 5.3.2 곱셈 노드의 역전파

$z = xy$ 라는 식이 있다면

$$ \frac{\partial{z}}{\partial{x}} = y $$  
$$ \frac{\partial{z}}{\partial{y}} = x $$

왼쪽이 순전파, 오른쪽이 역전파

<img width="651" alt="fig 5-12" src="https://user-images.githubusercontent.com/77653353/194430669-ae8cb860-49a4-4916-8e17-c8b3e28ab4b4.png">

위의 그림에서 상류에서 전해진 미분이 $\frac{\partial{L}}{\partial{z}}$이라고 한다면  
곱곱 노드의 역전파는 상류의 값에 순전파 때의 입력 신호들을 <code>서로 바꾼 값</code>을 곱해서 하류로 보내게 됨

덧셈의 역전파에서는 상류의 값을 그대로 흘려보내서 순방향 입력 신호의 값은 필요하지 않지만  
곱셈의 역전파에서는 순방향 입력 신호의 값이 필요하기에 곱셈 노드 구현 시 순전파의 입력 신호를 유지함

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        return dx, dy

### 5.4.2 덧셈 계층

In [3]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy